In [278]:
import pandas as pd
import numpy as np
import datetime as dt

def calculate_linearized_metric(
    df, value_name, user_id_name, list_user_id, date_name, period, metric_name, kappa=None
):
    """Вычисляет значение линеаризованной метрики для списка пользователей в определённый период.
    
    df - pd.DataFrame, датафрейм с данными
    value_name - str, название столбца со значениями для вычисления целевой метрики
    user_id_name - str, название столбца с идентификаторами пользователей
    list_user_id - List[int], список идентификаторов пользователей, для которых нужно посчитать метрики
    date_name - str, название столбца с датами
    period - dict, словарь с датами начала и конца периода, за который нужно посчитать метрики.
        Пример, {'begin': '2020-01-01', 'end': '2020-01-08'}. Дата начала периода входит в
        полуинтервал, а дата окончания нет, то есть '2020-01-01' <= date < '2020-01-08'.
    metric_name - str, название полученной метрики
    kappa - float, коэффициент в функции линеаризации.
        Если None, то посчитать как ratio метрику по имеющимся данным.

    return - pd.DataFrame, со столбцами [user_id_name, metric_name], кол-во строк должно быть равно
        кол-ву элементов в списке list_user_id.
    """

    # YOUR_CODE_HERE
    data = df.copy()
    data[date_name] = pd.to_datetime(data[date_name])
    mask = (period['begin'] <= data[date_name]) & (data[date_name] < period['end']) & (data[user_id_name].isin(list_user_id))
    
    data = data[mask]\
            .drop(columns=[date_name])\
            .groupby(user_id_name)\
            .agg(['sum','count'])\
            .reset_index()

    columns = [f"{a}_{b}" if user_id_name not in f"{a}" else user_id_name for a,b in data.columns]
    data.columns = columns

    x = data[f"{value_name}_sum"].to_numpy()
    y = data[f"{value_name}_count"].to_numpy()
    if kappa is None: kappa = x.sum()/y.sum()

    l = x - kappa * y

    data[metric_name] = l
    data = pd.DataFrame(data = np.array(list_user_id), columns=[user_id_name]).merge(data, how='outer', on=[user_id_name]).fillna(0)

    rc = pd.DataFrame()
    rc[user_id_name] = data[user_id_name]
    rc[metric_name]  = data[metric_name]

    return rc

In [243]:
#
# Генерируем тестовые данные
#

def generate_purchase_list(mean, num) ->np.array:
    """
    Генерирует список стомостей покупок
    mean - средняя стоиомсть покупки
    num  - число покупок, для которых нало сгенерировать стоимости
    выход - горизонтальнйы np.array, shape(1,)
    """
    purchase_list = np.random.normal(loc=mean, scale=mean/2, size=num).round(2)
    purchase_list = np.where(purchase_list > 0, purchase_list, 1)
    return purchase_list

seed = 152 #None
if seed is not None: np.random.seed(seed)

num_users = 5
start_date = dt.datetime(2020, 1, 1)
num_days = 4
mean_purchase_cost = 100                   # средняя стоимость одной покупки
max_purchases_x_user_x_day = 5             # max кол-во покупок пользователя/день
num_user_entries_x_day = 6                # кол-во заходов в магазин в день, один и тот же user может заходить несколько раз  

end_date = start_date + dt.timedelta(days=num_days)
dates_list = pd.date_range(min(start_date, end_date), max(start_date, end_date)).strftime('%Y-%m-%d').tolist()
user_ids = np.arange(1, num_users+1)

df = pd.DataFrame()
for this_day in dates_list:
    
    # генерируем cлучайный массив [user_id, num_purchases] для данного дня
    purchases_this_day = np.random.randint(low=[1,1]
                                           ,high=[num_users+1
                                           ,max_purchases_x_user_x_day+1]
                                           ,size=(num_user_entries_x_day,2))

    for [user_id_, num_purchases_this_day_] in purchases_this_day:
        purchase_values_ = np.reshape(
                            np.array(generate_purchase_list(mean = mean_purchase_cost
                                     ,num = num_purchases_this_day_))
                            , (-1,1))
        data_ = np.concatenate(
            (np.array( len(purchase_values_)*[[this_day, user_id_]] )
                       ,purchase_values_)
                       ,axis=1 )
    
    #print(f"{user_id_=}, len={len(purchase_values_)},\n{purchase_values_=},\n{data_=}")
    
    df_ = pd.DataFrame(data=data_, columns=['date', 'user_id', 'purchase'])
    
    df = pd.concat([df,df_])
    
df = df.reset_index().drop(columns=['index'])
df['user_id'] = df['user_id'].astype('int')
df['purchase'] = df['purchase'].astype('float64')

print(start_date, end_date)
df.head()

2020-01-01 00:00:00 2020-01-05 00:00:00


,date,user_id,purchase
0,2020-01-01,5,93.72
1,2020-01-01,5,96.84
2,2020-01-02,3,77.29
3,2020-01-02,3,116.74
4,2020-01-03,2,100.73


In [258]:
value_name = 'purchase'
user_id_name = 'user_id'
date_name = 'date'
metric_name = 'metric_value'

used_ids = df['user_id'].unique().tolist()
list_user_id = used_ids + [list(set(np.arange(1,num_users+1).tolist()) - set(used_ids))[0]]

period = {'begin': '2020-01-02', 'end': '2020-01-04'}
kappa=None

In [279]:
calculate_linearized_metric(
    df, value_name, user_id_name, list_user_id, date_name, period, metric_name, kappa=None
)

,user_id,metric_value
0,5,0.000000
1,3,-17.078571
2,2,17.078571
3,4,0.000000
4,1,0.000000


In [280]:
data = df.copy()
data[date_name] = pd.to_datetime(data[date_name])
mask = (period['begin'] <= data[date_name]) & (data[date_name] < period['end']) & (data[user_id_name].isin(list_user_id))
    
data = data[mask]\
            .drop(columns=[date_name])\
            .groupby(user_id_name)\
            .agg(['sum','count'])\
            .reset_index()

#columns = [f"{a}_{b}" if user_id_name not in f"{a}" else user_id_name for a,b in data.columns]
#data.columns = columns
data

user_id purchase      
               sum count
0       2   544.85     5
1       3   194.03     2

In [287]:
data.columns.get_level_values(1)

Index(['', 'sum', 'count'], dtype='object')

In [274]:
data = df.copy()
data[date_name] = pd.to_datetime(data[date_name])
mask = (period['begin'] <= data[date_name]) & (data[date_name] < period['end']) & (data[user_id_name].isin(list_user_id))
    
data = data[mask]\
            .drop(columns=[date_name])\
            .groupby(user_id_name)\
            .agg(['sum','count'])\
            .reset_index()

columns = [f"{a}_{b}" if user_id_name not in f"{a}" else user_id_name for a,b in data.columns]
data.columns = columns

x = data[f"{value_name}_sum"].to_numpy()
y = data[f"{value_name}_count"].to_numpy()
if kappa is None: kappa = x.sum()/y.sum()

l = x - kappa * y

data[metric_name] = l
data = pd.DataFrame(data = np.array(list_user_id), columns=[user_id_name]).merge(data, how='outer', on=[user_id_name]).fillna(0)

rc = pd.DataFrame()
rc[user_id_name] = data[user_id_name]
rc[metric_name]  = data[metric_name]

print(f"{x=}, {y=}, {kappa=}, {l=}")


data, rc

x=array([544.85, 194.03]), y=array([5, 2]), kappa=105.55428571428571, l=array([ 17.07857143, -17.07857143])


(   user_id  purchase_sum  purchase_count  metric_value
 0        5          0.00             0.0      0.000000
 1        3        194.03             2.0    -17.078571
 2        2        544.85             5.0     17.078571
 3        4          0.00             0.0      0.000000
 4        1          0.00             0.0      0.000000,
    user_id  metric_value
 0        5      0.000000
 1        3    -17.078571
 2        2     17.078571
 3        4      0.000000
 4        1      0.000000)

In [270]:
list_user_id


,user_id,purchase_sum,purchase_count,metric_value
0,5,0.00,0.0,0.000000
1,3,194.03,2.0,-17.078571
2,2,544.85,5.0,17.078571
3,4,0.00,0.0,0.000000
4,1,0.00,0.0,0.000000


In [267]:

data

,user_id_,purchase_sum,purchase_count,metric_value
0,2,544.85,5,17.078571
1,3,194.03,2,-17.078571


In [ ]:
# Генерация тестовых данных
def get_session_duration(size, effect=0, seed=None):
    """Генерирует данные с продолжительностями сессий.
    
    size: int, количество пользователей.
    effect: float, размер эффекта, на сколько изменилась продолжительность сессии относительно базовой.
    seed: int, состоянеие генератора случайных чисел
    
    return: List[np.array], список массивов,
        элемент списка - пользователь,
        элементы массива - длины сессий.
    """
    def _user_sessions(mean):
        size = np.random.randint(3,10)
        duration = np.random.normal(loc=mean, scale=10, size=size).round()
        duration = np.where(duration > 0, duration, 0)
        return duration
    
    if seed:
        np.random.seed(seed)
    
    mean_durations = np.random.normal(loc=100, scale=20, size=size) * (1 + effect)
    return [_user_sessions(mean) for mean in mean_durations]

In [4]:
dict_stats = {'a': {'data': [1,2,3]}, 'b': {'data': [4,5,6]}}
for key, dict_ in dict_stats.items():
    print(key, dict_)
    dict_['new']='added'
    print(key, dict_)

print(dict_stats)

a {'data': [1, 2, 3]}
a {'data': [1, 2, 3], 'new': 'added'}
b {'data': [4, 5, 6]}
b {'data': [4, 5, 6], 'new': 'added'}
{'a': {'data': [1, 2, 3], 'new': 'added'}, 'b': {'data': [4, 5, 6], 'new': 'added'}}
